# Setup the environment

In [1]:
import sys
sys.path.append('../../FourVector')
sys.path.append('../project')

from FourVector import FourVector
from ThreeVector import ThreeVector

from FutureColliderTools import SmearVertex, GetCorrectedMass, GetMissingMass2, GetQ2
from FutureColliderDataLoader import LoadData_KMuNu, LoadData_DsMuNu

from FutureColliderVariables import DataTuple, sigma_PV_LHCb, sigma_SV_LHCb

import numpy as np
import ROOT

ROOT.enableJSVis()
ROOT.gStyle.SetOptStat(0)



Welcome to JupyROOT 6.08/02


# Load $B_s \to K^- \mu^+ \nu_\mu$ Data

### Print a few events so the structure can be seen

In [2]:
from prettytable import PrettyTable

EventType = "13512010"
FileName, TreeName = DataTuple[ EventType ]

InputData, K_PE, Mu_PE, B_PE, B_Origin, B_End = LoadData_KMuNu(EventType, (FileName, TreeName) )

DataTable = PrettyTable()
DataTable.add_column("", InputData.dtype.names)
for x in range(4):
    DataTable.add_column("Ev {0}".format(x), InputData[x])
print DataTable

+-------------------------+---------------+---------------+---------------+---------------+
|                         |      Ev 0     |      Ev 1     |      Ev 2     |      Ev 3     |
+-------------------------+---------------+---------------+---------------+---------------+
|      Kminus_TRUEP_E     | 61977.3825059 | 43248.3310134 | 5457.04280306 | 9077.88373525 |
|      Kminus_TRUEP_X     |     552.99    |    -3780.28   |    -236.81    |    -824.94    |
|      Kminus_TRUEP_Y     |    -1903.15   |    2557.32    |    -1023.31   |     1381.0    |
|      Kminus_TRUEP_Z     |    61943.72   |    43004.0    |     5332.2    |    8920.57    |
|      muplus_TRUEP_E     | 163875.924011 | 960.302049387 | 71498.3097027 | 6061.79860869 |
|      muplus_TRUEP_X     |     -289.8    |     13.83     |    -758.05    |     107.97    |
|      muplus_TRUEP_Y     |    -1992.52   |    -114.07    |    -3986.38   |    -1484.0    |
|      muplus_TRUEP_Z     |   163863.52   |     947.53    |    71382.99   |     

# Organise the data into a nice format

* Generate FourVectors fo the kaon and muon
* Smear the primary and secondary vertices
* Generate a ThreeVector of the B direction

In [3]:
K  = FourVector( InputData[K_PE ] )
Mu = FourVector( InputData[Mu_PE] )

Y = K+Mu

DataType = InputData[B_End[0]].dtype
Smeared_SV = SmearVertex( InputData[B_End]   .copy().view( (DataType, 3) ), sigma_SV_LHCb )
Smeared_PV = SmearVertex( InputData[B_Origin].copy().view( (DataType, 3) ), sigma_PV_LHCb )

B_Dir = ThreeVector( Smeared_PV ) - ThreeVector( Smeared_SV )


# Do some nice things with the data
* generate the corrected mass of the events
* generated the missing mass squared of the events
* generate the two quadratic solutions to the $q^2$

In [4]:
MCORR        = GetCorrectedMass( Y, B_Dir)
MissingM2    = GetMissingMass2(K, Mu, B_Dir)
Qsq_1, Qsq_2 = GetQ2(Y, Mu, B_Dir )

IndexError: invalid index to scalar variable.

Trying 1
Trying 2
(<ThreeVector.ThreeVector instance at 0x7f8d7312f2d8>, 5366.0)
Trying 3
Trying 1
Trying 2
None
Trying 3
5366.0 0 0


# Plot the data

In [ ]:
h_MCORR = ROOT.TH1F("h_MCORR", "B_{s} Corrected Mass", 100, 3000, 6000)
h_MM2   = ROOT.TH1F("h_MM2"  , "Missing Mass Squared", 100, -4e6, 12e6)
h_Qsq_1 = ROOT.TH1F("h_QSQ1" , "q^{2} Quadratic Solutions", 100, 0, 24e6)
h_Qsq_2 = ROOT.TH1F("h_QSQ2" , "q^{2} Quadratic Solutions", 100, 0, 24e6)

nev = len(K)

h_MCORR.FillN(nev, MCORR,     np.ones(nev))
h_MM2  .FillN(nev, MissingM2, np.ones(nev))
h_Qsq_1.FillN(nev, Qsq_1,     np.ones(nev))
h_Qsq_2.FillN(nev, Qsq_2,     np.ones(nev))

canvas_1 = ROOT.TCanvas("c1", "c1", 900,300)
canvas_1.Divide(3)
canvas_1.cd(1)
h_MCORR.Draw()
canvas_1.cd(2)
h_MM2.Draw()
canvas_1.cd(3)
h_Qsq_1.Draw()
h_Qsq_2.Draw("SAME")
canvas_1.Draw()